# データを確認する

- csvデータをDataFrame化して、特徴を把握する
- 分析の方針を立てる
- データの加工方法を決定し、分析用のDataFrameを作成する

In [37]:
import pandas as pd
from pandas import DataFrame

## air_reserve.csv [Airリザーブの予約データ]

- 92,378レコード
- 314 /829店舗分の予約データ
- 訪問予定は、2016-01-01 19:00:00 〜 2017-05-31 21:00:00
- 予約時刻は、2016-01-01 01:00:00 〜 2017-04-22 23:00:00

In [87]:
df1 = pd.read_csv('Data/csv/air_reserve.csv')
df1.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5


In [3]:
df1.describe()

,reserve_visitors
count,92378.000000
mean,4.481749
std,4.919669
min,1.000000
25%,2.000000
50%,3.000000
75%,5.000000
max,100.000000


In [4]:
df1_id = df1.air_store_id.value_counts().count()
df1_min_vdate = df1.visit_datetime.min()
df1_max_vdate = df1.visit_datetime.max()
df1_min_rdate = df1.reserve_datetime.min()
df1_max_rdate = df1.reserve_datetime.max()
print(df1_id,df1_min_vdate,df1_max_vdate,df1_min_rdate,df1_max_rdate)

314 2016-01-01 19:00:00 2017-05-31 21:00:00 2016-01-01 01:00:00 2017-04-22 23:00:00



## air_store_info.csv [Airリザーブのレストラン情報]

- 829店舗
- 14ジャンル
- 103エリア

In [5]:
df2 = pd.read_csv('Data/csv/air_store_info.csv')
df2.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


In [6]:
df2.describe()

,latitude,longitude
count,829.000000,829.000000
mean,35.647042,137.415311
std,2.084659,3.650971
min,33.211967,130.195555
25%,34.695124,135.341564
50%,35.658068,139.685474
75%,35.694003,139.751599
max,44.020632,144.273398


In [7]:
df2_genre = df2.air_genre_name.value_counts().count()
df2_area = df2.air_area_name.value_counts().count()
print(df2_genre,df2_area)

14 103


In [8]:
df2.groupby('air_genre_name').size()

air_genre_name
Asian                             2
Bar/Cocktail                     79
Cafe/Sweets                     181
Creative cuisine                 13
Dining bar                      108
International cuisine             2
Italian/French                  102
Izakaya                         197
Japanese food                    63
Karaoke/Party                     2
Okonomiyaki/Monja/Teppanyaki     14
Other                            27
Western food                     16
Yakiniku/Korean food             23
dtype: int64

In [9]:
df2.groupby('air_area_name').size()

air_area_name
Fukuoka-ken Fukuoka-shi Daimyō                   64
Fukuoka-ken Fukuoka-shi Hakata Ekimae            16
Fukuoka-ken Fukuoka-shi Imaizumi                  2
Fukuoka-ken Fukuoka-shi Momochi                   6
Fukuoka-ken Fukuoka-shi Shiobaru                  7
Fukuoka-ken Fukuoka-shi Takatori                  2
Fukuoka-ken Fukuoka-shi Tenjin                    2
Fukuoka-ken Fukuoka-shi Torikai                   4
Fukuoka-ken Itoshima-shi Maebarunishi             7
Fukuoka-ken Kitakyūshū-shi Konyamachi             2
Fukuoka-ken Kitakyūshū-shi None                   3
Fukuoka-ken Kitakyūshū-shi Ōtemachi               4
Fukuoka-ken Kurume-shi Jōnanmachi                 6
Fukuoka-ken Yame-shi Motomachi                    2
Hiroshima-ken Fukuyama-shi Higashisakuramachi     7
Hiroshima-ken Hiroshima-shi Kokutaijimachi       23
Hiroshima-ken Hiroshima-shi Nagarekawachō         2
Hokkaidō Abashiri-shi Minami 6 Jōhigashi          2
Hokkaidō Asahikawa-shi 6 Jōdōri                  1

## air_visit_data.csv [Airリザーブの訪問データ]

- 252,108レコード
- 829店舗全ての履歴データがある

In [73]:
df3 = pd.read_csv('Data/csv/air_visit_data.csv')
df3.head()

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


In [11]:
df3.describe()

,visitors
count,252108.000000
mean,20.973761
std,16.757007
min,1.000000
25%,9.000000
50%,17.000000
75%,29.000000
max,877.000000


In [12]:
df3.air_store_id.value_counts().count()

829

## hpg_reserve.csv [ホットペッパーグルメの予約データ]

- 2,000,320レコード
- 13,325 /4,690店舗の予約データ???
- 訪問予定は、2016-01-01 11:00:00 〜 2017-05-31 23:00:00
- 予約時刻は、2016-01-01 00:00:00 〜 2017-04-22 23:00:00

In [13]:
df4 = pd.read_csv('Data/csv/hpg_reserve.csv')
df4.head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13


In [14]:
df4.describe()

,reserve_visitors
count,2.000320e+06
mean,5.073785e+00
std,5.416172e+00
min,1.000000e+00
25%,2.000000e+00
50%,3.000000e+00
75%,6.000000e+00
max,1.000000e+02


In [15]:
df4_records = df4.hpg_store_id.count()
df4_id = df4.hpg_store_id.value_counts().count()
df4_min_vdate = df4.visit_datetime.min()
df4_max_vdate = df4.visit_datetime.max()
df4_min_rdate = df4.reserve_datetime.min()
df4_max_rdate = df4.reserve_datetime.max()
print(df4_records,df4_id,df4_min_vdate,df4_max_vdate,df4_min_rdate,df4_max_rdate)

2000320 13325 2016-01-01 11:00:00 2017-05-31 23:00:00 2016-01-01 00:00:00 2017-04-22 23:00:00


## hpg_store_info.csv [ホットペッパーグルメのレストラン情報]

- 4,690の店舗データ
- 34ジャンル
- 119エリア

In [16]:
df5 = pd.read_csv('Data/csv/hpg_store_info.csv')
df5.head()

,hpg_store_id,hpg_genre_name,hpg_area_name,latitude,longitude
0,hpg_6622b62385aec8bf,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
1,hpg_e9e068dd49c5fa00,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
2,hpg_2976f7acb4b3a3bc,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
3,hpg_e51a522e098f024c,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
4,hpg_e3d0e1519894f275,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221


In [17]:
df5.describe()

,latitude,longitude
count,4690.000000,4690.000000
mean,35.810261,137.675816
std,2.138755,3.197233
min,33.311645,130.339313
25%,34.692109,135.498859
50%,35.659214,139.495733
75%,35.703381,139.737998
max,43.774846,143.714585


In [18]:
df5_id = df5.hpg_store_id.value_counts().count()
df5_genre = df5.hpg_genre_name.value_counts().count()
df5_area = df5.hpg_area_name.value_counts().count()
print(df5_id,df5_genre,df5_area)

4690 34 119


In [19]:
df5.groupby('hpg_genre_name').size()

hpg_genre_name
Amusement bar                      5
Bar/Cocktail                       7
Bistro                            22
Cafe                              16
Cantonese food                     4
Chinese general                   91
Creation                         410
Creative Japanese food            60
Dim Sum/Dumplings                  2
French                            27
Grilled meat                     325
International cuisine            700
Italian                          249
Japanese cuisine/Kaiseki          64
Japanese food in general          85
Japanese style                  1750
Karaoke                           60
Korean cuisine                    38
Okonomiyaki/Monja/Teppanyaki      44
Party                             40
Pasta/Pizza                       10
Seafood                          339
Shabu-shabu/Sukiyaki              59
Shanghai food                      1
Sichuan food                       3
Spain Bar/Italian Bar            229
Spain/Mediterranean cui

In [20]:
df5.groupby('hpg_area_name').size()

hpg_area_name
Fukuoka-ken Chikushino-shi Yamaguchi                               5
Fukuoka-ken Fukuoka-shi Daimyō                                    50
Fukuoka-ken Fukuoka-shi Hakata Ekimae                             66
Fukuoka-ken Fukuoka-shi Matsushima                                 3
Fukuoka-ken Fukuoka-shi Minamishō                                 22
Fukuoka-ken Fukuoka-shi None                                      90
Fukuoka-ken Fukuoka-shi Ōmiya                                     26
Fukuoka-ken Iizuka-shi Mashiki                                     2
Fukuoka-ken Kitakyūshū-shi Aoyama                                 19
Fukuoka-ken Kitakyūshū-shi Sunatsu                                65
Fukuoka-ken Kurume-shi Higashimachi                               32
Hiroshima-ken Fukuyama-shi Sannomaruchō                           40
Hiroshima-ken Higashihiroshima-shi Hachihonmatsuchō Yoshikawa     10
Hiroshima-ken Hiroshima-shi Hondōri                              135
Hiroshima-ken Hirosh

## date_info.csv [カレンダー]

- 2016-01-01〜2017-05-31

In [21]:
df6 = pd.read_csv('Data/csv/date_info.csv')
df6.head()

,calendar_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


In [22]:
df6_min_date = df6.calendar_date.min()
df6_max_date = df6.calendar_date.max()
print(df6_min_date,df6_max_date)

2016-01-01 2017-05-31


## store_id_relation.csv [id横断データ]

- 150店舗、１対１

In [23]:
df7 = pd.read_csv('Data/csv/store_id_relation.csv')
df7.head()

,air_store_id,hpg_store_id
0,air_63b13c56b7201bd9,hpg_4bc649e72e2a239a
1,air_a24bf50c3e90d583,hpg_c34b496d0305a809
2,air_c7f78b4f3cba33ff,hpg_cd8ae0d9bbd58ff9
3,air_947eb2cae4f3e8f2,hpg_de24ea49dc25d6b8
4,air_965b2e0cf4119003,hpg_653238a84804d8e7


In [24]:
df7.describe()

,air_store_id,hpg_store_id
count,150,150
unique,150,150
top,air_ee3ba9af184c6c82,hpg_de24ea49dc25d6b8
freq,1,1


## sample_submission.csv [Submitサンプル]

- store_idに、アンダースコア、訪問日を付与する必要がある
- 2017-04-23〜2017-05-31が予測対象期間
- visitorsに予測値を出力する

In [25]:
sample = pd.read_csv('Data/csv/sample_submission.csv')
sample.head()

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,0


### メモ
- 大口の予約レコードの取り扱い方法
- 予約に対する実訪問データの状況
- ジャンルのマッピングが必要そう
- 2016年4,5月の実績が重要。それ以外の期間の重み付けを下げる方法はあるのか...?

- 天候 [予測対象期間が過去のため]
- 歓迎会シーズン
- オフィス数、住宅数 [都市では活きないか？]
- イベント[祭りなど]
- 人口
- 飲食店の数


# 訪問数に対する予約率を見てみる

In [85]:
df1.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5


In [88]:
df1['visit_datetime'] = df1['visit_datetime'].str[:10]
df1.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01,2016-01-01 16:00:00,1
1,air_db4b38ebe7a7ceff,2016-01-01,2016-01-01 19:00:00,3
2,air_db4b38ebe7a7ceff,2016-01-01,2016-01-01 19:00:00,6
3,air_877f79706adbfb06,2016-01-01,2016-01-01 16:00:00,2
4,air_db80363d35f10926,2016-01-01,2016-01-01 01:00:00,5


In [112]:
df1_edit = df1.rename(columns={'visit_datetime': 'visit_date'}).drop('reserve_datetime',axis=1)
df1_edit = df1_edit.groupby(['air_store_id','visit_date'])['reserve_visitors'].sum().reset_index()
df1_edit.head()

,air_store_id,visit_date,reserve_visitors
0,air_00a91d42b08b08d9,2016-10-31,2
1,air_00a91d42b08b08d9,2016-12-05,9
2,air_00a91d42b08b08d9,2016-12-14,18
3,air_00a91d42b08b08d9,2016-12-17,2
4,air_00a91d42b08b08d9,2016-12-20,4


In [99]:
df3.head()

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


In [113]:
air_df = pd.merge(df3,df1_edit,on=['air_store_id','visit_date'],how='left')
air_df.head()

,air_store_id,visit_date,visitors,reserve_visitors
0,air_ba937bf13d40fb24,2016-01-13,25,NaN
1,air_ba937bf13d40fb24,2016-01-14,32,NaN
2,air_ba937bf13d40fb24,2016-01-15,29,NaN
3,air_ba937bf13d40fb24,2016-01-16,22,NaN
4,air_ba937bf13d40fb24,2016-01-18,6,NaN


In [114]:
air_df.columns

Index(['air_store_id', 'visit_date', 'visitors', 'reserve_visitors'], dtype='object')

In [117]:
air_df = DataFrame(air_df, columns=['air_store_id','visit_date','visitors','reserve_visitors','reserve_rate'])
air_df['reserve_rate'] = air_df['reserve_visitors'] / air_df['visitors'] * 100
air_df.head()

,air_store_id,visit_date,visitors,reserve_visitors,reserve_rate
0,air_ba937bf13d40fb24,2016-01-13,25,NaN,NaN
1,air_ba937bf13d40fb24,2016-01-14,32,NaN,NaN
2,air_ba937bf13d40fb24,2016-01-15,29,NaN,NaN
3,air_ba937bf13d40fb24,2016-01-16,22,NaN,NaN
4,air_ba937bf13d40fb24,2016-01-18,6,NaN,NaN


In [118]:
air_df[air_df['reserve_rate'] > 0]

,air_store_id,visit_date,visitors,reserve_visitors,reserve_rate
1006,air_35512c42db0868da,2016-02-27,6,12.0,200.000000
3744,air_ee3a01f0c71a769f,2016-01-04,61,2.0,3.278689
3747,air_ee3a01f0c71a769f,2016-01-08,21,3.0,14.285714
3748,air_ee3a01f0c71a769f,2016-01-09,57,25.0,43.859649
3749,air_ee3a01f0c71a769f,2016-01-10,32,5.0,15.625000
3750,air_ee3a01f0c71a769f,2016-01-11,35,6.0,17.142857
3754,air_ee3a01f0c71a769f,2016-01-16,34,6.0,17.647059
3755,air_ee3a01f0c71a769f,2016-01-17,58,12.0,20.689655
3756,air_ee3a01f0c71a769f,2016-01-18,14,2.0,14.285714
3758,air_ee3a01f0c71a769f,2016-01-22,6,1.0,16.666667
